In [1]:
import random, gc, os, pickle, csv, time, re

import datasets.utils
import models.utils
from models.cls_oml_ori_v2 import OML
from models.base_models_ori import LabelAwareReplayMemory

import numpy as np

import higher
import torch
import torch.nn.functional as F
from torch.utils import data

# Ablation 1: No Sort Adapt Score
This Ablation focuses on no sorting of adapt score. So during query should be random sort.

There are a total of 4 changes:
1. Training cls_oml_ori_v2.py: Random Score on Sample Support (no_support_priority=False)
2. Training cls_oml_ori_v2.py: No memory write of update adapt score (memory.update_meta)
3. Memory base_models_ori.py: Write filter condition don’t care (sorted_support >= 1)
4. Testing: No Sample Score (sort_score=False)


# Constants

In [2]:
dataset_order_mapping = {
    1: [2, 0, 3, 1, 4],
    2: [3, 4, 0, 1, 2],
    3: [2, 4, 1, 3, 0],
    4: [0, 2, 1, 4, 3]
}
n_classes = 33
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')


BASE_MODEL_PATH = "/data/model_runs/original_oml/aMEL-allquery-order4"
find_path = "id4"

model_name = [x for x in os.listdir(BASE_MODEL_PATH) if  re.search(f"{find_path}.*\.pt$", x)][0]
model_path = os.path.join(BASE_MODEL_PATH, model_name)
memory_name = [x for x in os.listdir(BASE_MODEL_PATH) if  re.search(f"{find_path}.*memory\.pickle$", x)][0]
memory_path = os.path.join(BASE_MODEL_PATH, memory_name)
print(f"Using model from {model_path}")
print(f"Using memory_path from {memory_path}")

use_db_cache = True
cache_dir = 'tmp'

Using model from /data/model_runs/original_oml/aMEL-allquery-order4/OML-order4-id4-2023-10-02_08-07-55.558316.pt
Using memory_path from /data/model_runs/original_oml/aMEL-allquery-order4/OML-order4-id4-2023-10-02_08-07-55.558350_memory.pickle


In [3]:
args = {
    "order": 1,
    "n_epochs": 1,
    "lr": 3e-5,
    "inner_lr": 0.001*5,
    "meta_lr": 3e-5,
    "model": "roberta",
    "learner": "oml",
    "mini_batch_size": 16,
    "updates": 5*0,
    "write_prob": 0.1,
    "max_length": 448,
    "seed": 42,
    "replay_rate": 0.01,
    "replay_every": 8000,
    "task_aware": True,
    "reverse_support": True,
    "curriculum_replay": True,
    "pln": "1fc"
}
sort_score = False # This will be false for ablation!!
updates = args["updates"]
mini_batch_size = args["mini_batch_size"]
order = args["order"]

In [4]:
torch.manual_seed(args["seed"])
random.seed(args["seed"])
np.random.seed(args["seed"])

# Load Dataset

In [5]:
print('Loading the datasets')
test_datasets = []
for dataset_id in dataset_order_mapping[order]:
    test_dataset_file = os.path.join(cache_dir, f"{dataset_id}.cache")
    if os.path.exists(test_dataset_file):
        with open(test_dataset_file, 'rb') as f:
            test_dataset = pickle.load(f)
    else:
        test_dataset = datasets.utils.get_dataset_test("", dataset_id)
        print('Loaded {}'.format(test_dataset.__class__.__name__))
        test_dataset = datasets.utils.offset_labels(test_dataset)
        pickle.dump(test_dataset, open( test_dataset_file, "wb" ), protocol=pickle.HIGHEST_PROTOCOL)
        print(f"Pickle saved at {test_dataset_file}")
    test_datasets.append(test_dataset)
print('Finished loading all the datasets')

Loading the datasets
Finished loading all the datasets


# Load Model

In [6]:
learner = OML(device=device, n_classes=n_classes, **args)
print('Using {} as learner'.format(learner.__class__.__name__))
learner.load_model(model_path)
with open(memory_path, 'rb') as f:
#     learner.memory = pickle.load(f)
    memory_buffer = pickle.load(f)


2023-11-12 19:41:37,964 - transformers.tokenization_utils_base - INFO - loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-vocab.json from cache at /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
2023-11-12 19:41:37,967 - transformers.tokenization_utils_base - INFO - loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-merges.txt from cache at /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
2023-11-12 19:41:39,274 - transformers.configuration_utils - INFO - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c2

Using OML as learner


In [7]:
# Setting up task dict for task-aware
memory_buffer.task_dict = {
    0: list(range(5, 9)), # AG
    1: list(range(0, 5)), # Amazon
    2: list(range(0, 5)), # Yelp
    3: list(range(9, 23)), # DBPedia
    4: list(range(23, 33)), # Yahoo
}

In [8]:
dataclass_mapper = {
    "AGNewsDataset": 0,
    "AmazonDataset": 1,
    "YelpDataset": 2,
    "DBPediaDataset": 3,
    "YahooAnswersDataset": 4
}
dataclass_mapper["AGNewsDataset"]

0

# Testing

Select specific column index per row
https://stackoverflow.com/questions/23435782/numpy-selecting-specific-column-index-per-row-by-using-a-list-of-indexes

In [9]:
def evaluate(dataloader, updates, mini_batch_size, dataname=""):
    learner.rln.eval()
    learner.pln.train()
    
    all_losses, all_predictions, all_labels, all_label_conf = [], [], [], []
    all_adaptation_time = []
    # Get Query set first. and then find supporting support set
    for query_idx, (query_text, query_labels) in enumerate(dataloader):
        print(f"Query ID {query_idx}/{len(dataloader)}")
        # The task id to optimize to for support set
        # task_idx = get_task_from_label_list(query_labels, memory_buffer.task_dict)
        task_idx = dataclass_mapper[dataname]
        
    
        support_set = []
        for _ in range(updates):
            text, labels = memory_buffer.read_batch(batch_size=mini_batch_size)
            support_set.append((text, labels))

        with higher.innerloop_ctx(learner.pln, learner.inner_optimizer,
                                  copy_initial_weights=False, track_higher_grads=False) as (fpln, diffopt):
            
            INNER_tic = time.time()
            # Inner loop
            task_predictions, task_labels = [], []
            support_loss = []
            for text, labels in support_set:
                labels = torch.tensor(labels).to(device)
                input_dict = learner.rln.encode_text(text)
                _repr = learner.rln(input_dict)
                output = fpln(_repr)
                loss = learner.loss_fn(output, labels)
                diffopt.step(loss)
                pred = models.utils.make_prediction(output.detach())
                support_loss.append(loss.item())
                task_predictions.extend(pred.tolist())
                task_labels.extend(labels.tolist())
            INNER_toc = time.time() - INNER_tic
            all_adaptation_time.append(INNER_toc)

            if updates > 0: # You can remove this if, but then it's just ugly errors.
                acc, prec, rec, f1 = models.utils.calculate_metrics(task_predictions, task_labels)
                print('Support set metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, '
                            'recall = {:.4f}, F1 score = {:.4f}'.format(np.mean(support_loss), acc, prec, rec, f1))

            # Query set is now here!
            query_labels = torch.tensor(query_labels).to(device)
            query_input_dict = learner.rln.encode_text(query_text)
            with torch.no_grad():
                query_repr = learner.rln(query_input_dict)
                query_output = fpln(query_repr) # Output has size of torch.Size([16, 33]) [BATCH, CLASSES]
                query_loss = learner.loss_fn(query_output, query_labels)
            query_loss = query_loss.item()
            # print(output.detach().size())
            # output.detach().max(-1) max on each Batch, which will return [0] max, [1] indices
            query_output_softmax = F.softmax(query_output, -1)
            query_label_conf = query_output_softmax[np.arange(len(query_output_softmax)), query_labels] # Select labels in the softmax of 33 classes

            query_pred = models.utils.make_prediction(query_output.detach())
            query_acc, query_prec, query_rec, query_f1 = models.utils.calculate_metrics(query_pred.tolist(), query_labels.tolist())
            
            print('Query set metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, '
                'recall = {:.4f}, F1 score = {:.4f}'.format(np.mean(query_loss), query_acc, query_prec, query_rec, query_f1))

            all_losses.append(query_loss)
            all_predictions.extend(query_pred.tolist())
            all_labels.extend(query_labels.tolist())
            all_label_conf.extend(query_label_conf.tolist())

    acc, prec, rec, f1 = models.utils.calculate_metrics(all_predictions, all_labels)
    print('Test metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
                'F1 score = {:.4f}'.format(np.mean(all_losses), acc, prec, rec, f1))
    return acc, prec, rec, f1, all_predictions, all_labels, all_label_conf, all_adaptation_time

In [10]:
tic = time.time()
print('----------Testing on test set starts here----------')

accuracies, precisions, recalls, f1s = [], [], [], []
all_adapt_time = []
# Data for Visualization: [data_idx, label, label_conf, pred]
data_for_visual = []

for test_dataset in test_datasets:
    print('Testing on {}'.format(test_dataset.__class__.__name__))
    test_dataloader = data.DataLoader(test_dataset, batch_size=mini_batch_size, shuffle=False,
                                      collate_fn=datasets.utils.batch_encode)
    acc, prec, rec, f1, all_pred, all_label, all_label_conf, all_adaptation_time = evaluate(dataloader=test_dataloader, updates=updates, 
                                                mini_batch_size=mini_batch_size, dataname=test_dataset.__class__.__name__)
    
    data_ids = [test_dataset.__class__.__name__ + str(i) for i in range(len(all_label))]
    data_for_visual.extend(list(zip(data_ids, all_label, all_label_conf, all_pred)))
    all_adapt_time.extend(all_adaptation_time)
#     print(data_ids)
#     print(all_label)
#     raise Exception("BREAKPOINT")
    
    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1s.append(f1)


print()
print("COPY PASTA - not really but ok")
for row in accuracies:
    print(row)
print()
print('Overall test metrics: Accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
            'F1 score = {:.4f}'.format(np.mean(accuracies), np.mean(precisions), np.mean(recalls), np.mean(f1s)))

toc = time.time() - tic
print(f"Total Time used: {toc//60} minutes")

----------Testing on test set starts here----------
Testing on YelpDataset
Query ID 0/475
Query set metrics: Loss = 1.4999, accuracy = 0.5625, precision = 0.5833, recall = 0.5867, F1 score = 0.5810
Query ID 1/475
Query set metrics: Loss = 1.0759, accuracy = 0.7500, precision = 0.7267, recall = 0.7933, F1 score = 0.7200
Query ID 2/475
Query set metrics: Loss = 2.1624, accuracy = 0.5625, precision = 0.4700, recall = 0.5500, F1 score = 0.4990
Query ID 3/475
Query set metrics: Loss = 2.9879, accuracy = 0.4375, precision = 0.4167, recall = 0.4667, F1 score = 0.4014
Query ID 4/475
Query set metrics: Loss = 2.4026, accuracy = 0.3750, precision = 0.2800, recall = 0.3200, F1 score = 0.2891
Query ID 5/475
Query set metrics: Loss = 1.9726, accuracy = 0.4375, precision = 0.5300, recall = 0.3867, F1 score = 0.4071
Query ID 6/475
Query set metrics: Loss = 1.6467, accuracy = 0.5000, precision = 0.3700, recall = 0.4167, F1 score = 0.3905
Query ID 7/475
Query set metrics: Loss = 2.4475, accuracy = 0.50

Query set metrics: Loss = 0.9919, accuracy = 0.7500, precision = 0.7833, recall = 0.7833, F1 score = 0.7576
Query ID 67/475
Query set metrics: Loss = 0.9101, accuracy = 0.6875, precision = 0.8000, recall = 0.6625, F1 score = 0.7056
Query ID 68/475
Query set metrics: Loss = 1.8292, accuracy = 0.6250, precision = 0.7133, recall = 0.7133, F1 score = 0.6667
Query ID 69/475
Query set metrics: Loss = 1.0457, accuracy = 0.6250, precision = 0.7333, recall = 0.6833, F1 score = 0.6186
Query ID 70/475
Query set metrics: Loss = 2.4986, accuracy = 0.3750, precision = 0.4400, recall = 0.5200, F1 score = 0.4400
Query ID 71/475
Query set metrics: Loss = 1.1108, accuracy = 0.6875, precision = 0.6833, recall = 0.7333, F1 score = 0.6690
Query ID 72/475
Query set metrics: Loss = 0.9729, accuracy = 0.7500, precision = 0.7700, recall = 0.8300, F1 score = 0.7778
Query ID 73/475
Query set metrics: Loss = 2.4487, accuracy = 0.5625, precision = 0.4833, recall = 0.5233, F1 score = 0.4819
Query ID 74/475
Query se

Query set metrics: Loss = 1.1316, accuracy = 0.6250, precision = 0.6533, recall = 0.5933, F1 score = 0.5755
Query ID 133/475
Query set metrics: Loss = 1.4169, accuracy = 0.6250, precision = 0.4933, recall = 0.5433, F1 score = 0.5100
Query ID 134/475
Query set metrics: Loss = 1.1279, accuracy = 0.6875, precision = 0.7600, recall = 0.7433, F1 score = 0.6667
Query ID 135/475
Query set metrics: Loss = 0.7420, accuracy = 0.8750, precision = 0.8833, recall = 0.9000, F1 score = 0.8743
Query ID 136/475
Query set metrics: Loss = 2.4777, accuracy = 0.5000, precision = 0.4286, recall = 0.4457, F1 score = 0.3478
Query ID 137/475
Query set metrics: Loss = 1.5558, accuracy = 0.6875, precision = 0.7333, recall = 0.7333, F1 score = 0.6600
Query ID 138/475
Query set metrics: Loss = 0.9996, accuracy = 0.7500, precision = 0.6433, recall = 0.6833, F1 score = 0.6540
Query ID 139/475
Query set metrics: Loss = 1.0520, accuracy = 0.6875, precision = 0.6250, recall = 0.6250, F1 score = 0.6143
Query ID 140/475


Query set metrics: Loss = 1.1706, accuracy = 0.6250, precision = 0.6700, recall = 0.6867, F1 score = 0.5933
Query ID 199/475
Query set metrics: Loss = 1.0625, accuracy = 0.7500, precision = 0.8000, recall = 0.8133, F1 score = 0.7410
Query ID 200/475
Query set metrics: Loss = 1.7997, accuracy = 0.5000, precision = 0.4867, recall = 0.4867, F1 score = 0.4533
Query ID 201/475
Query set metrics: Loss = 2.0421, accuracy = 0.3750, precision = 0.4833, recall = 0.4633, F1 score = 0.3854
Query ID 202/475
Query set metrics: Loss = 2.5582, accuracy = 0.3750, precision = 0.3033, recall = 0.3667, F1 score = 0.3224
Query ID 203/475
Query set metrics: Loss = 0.9961, accuracy = 0.5625, precision = 0.4167, recall = 0.3976, F1 score = 0.4064
Query ID 204/475
Query set metrics: Loss = 1.7764, accuracy = 0.5625, precision = 0.5400, recall = 0.6100, F1 score = 0.5489
Query ID 205/475
Query set metrics: Loss = 2.6093, accuracy = 0.4375, precision = 0.4800, recall = 0.4633, F1 score = 0.4314
Query ID 206/475


Query set metrics: Loss = 1.2257, accuracy = 0.6875, precision = 0.6333, recall = 0.7429, F1 score = 0.6410
Query ID 265/475
Query set metrics: Loss = 0.8373, accuracy = 0.7500, precision = 0.7000, recall = 0.7600, F1 score = 0.7054
Query ID 266/475
Query set metrics: Loss = 1.5961, accuracy = 0.7500, precision = 0.6600, recall = 0.6933, F1 score = 0.6489
Query ID 267/475
Query set metrics: Loss = 2.3637, accuracy = 0.3750, precision = 0.3800, recall = 0.3733, F1 score = 0.3571
Query ID 268/475
Query set metrics: Loss = 1.2829, accuracy = 0.6875, precision = 0.7643, recall = 0.6833, F1 score = 0.6769
Query ID 269/475
Query set metrics: Loss = 0.9654, accuracy = 0.6875, precision = 0.7000, recall = 0.6333, F1 score = 0.6267
Query ID 270/475
Query set metrics: Loss = 0.8752, accuracy = 0.6250, precision = 0.6667, recall = 0.4810, F1 score = 0.5558
Query ID 271/475
Query set metrics: Loss = 1.9016, accuracy = 0.5625, precision = 0.5143, recall = 0.4933, F1 score = 0.4876
Query ID 272/475


...

Query set metrics: Loss = 0.9545, accuracy = 0.7500, precision = 0.6619, recall = 0.6786, F1 score = 0.6399
Query ID 153/475
Query set metrics: Loss = 0.4161, accuracy = 0.7500, precision = 0.6250, recall = 0.7708, F1 score = 0.6625
Query ID 154/475
Query set metrics: Loss = 0.1345, accuracy = 0.9375, precision = 1.0000, recall = 0.9375, F1 score = 0.9583
Query ID 155/475
Query set metrics: Loss = 0.9793, accuracy = 0.6875, precision = 0.7571, recall = 0.6429, F1 score = 0.6746
Query ID 156/475
Query set metrics: Loss = 1.4511, accuracy = 0.6875, precision = 0.7111, recall = 0.6389, F1 score = 0.6472
Query ID 157/475
Query set metrics: Loss = 0.6930, accuracy = 0.7500, precision = 0.7292, recall = 0.7188, F1 score = 0.7196
Query ID 158/475
Query set metrics: Loss = 0.6772, accuracy = 0.8125, precision = 0.7619, recall = 0.7738, F1 score = 0.7367
Query ID 159/475
Query set metrics: Loss = 1.5877, accuracy = 0.6250, precision = 0.5521, recall = 0.6458, F1 score = 0.5613
Query ID 160/475


Query set metrics: Loss = 1.3249, accuracy = 0.6250, precision = 0.5625, recall = 0.5417, F1 score = 0.5250
Query ID 219/475
Query set metrics: Loss = 1.3006, accuracy = 0.6875, precision = 0.7024, recall = 0.7619, F1 score = 0.6714
Query ID 220/475
Query set metrics: Loss = 1.5240, accuracy = 0.6250, precision = 0.7188, recall = 0.7604, F1 score = 0.6833
Query ID 221/475
Query set metrics: Loss = 1.4246, accuracy = 0.5625, precision = 0.6667, recall = 0.6071, F1 score = 0.6177
Query ID 222/475
Query set metrics: Loss = 0.7052, accuracy = 0.7500, precision = 0.7188, recall = 0.7292, F1 score = 0.6696
Query ID 223/475
Query set metrics: Loss = 2.0743, accuracy = 0.6250, precision = 0.6574, recall = 0.6111, F1 score = 0.5778
Query ID 224/475
Query set metrics: Loss = 0.9552, accuracy = 0.7500, precision = 0.7083, recall = 0.6250, F1 score = 0.6417
Query ID 225/475
Query set metrics: Loss = 1.6899, accuracy = 0.5625, precision = 0.8095, recall = 0.6286, F1 score = 0.6721
Query ID 226/475


Query set metrics: Loss = 1.9622, accuracy = 0.5625, precision = 0.6875, recall = 0.6354, F1 score = 0.6042
Query ID 285/475
Query set metrics: Loss = 1.4166, accuracy = 0.6875, precision = 0.5381, recall = 0.6143, F1 score = 0.5627
Query ID 286/475
Query set metrics: Loss = 1.3308, accuracy = 0.6875, precision = 0.5833, recall = 0.6500, F1 score = 0.5767
Query ID 287/475
Query set metrics: Loss = 1.1896, accuracy = 0.6250, precision = 0.8214, recall = 0.5595, F1 score = 0.6500
Query ID 288/475
Query set metrics: Loss = 0.1544, accuracy = 0.9375, precision = 0.8333, recall = 0.8889, F1 score = 0.8519
Query ID 289/475
Query set metrics: Loss = 1.1737, accuracy = 0.6875, precision = 0.7500, recall = 0.7500, F1 score = 0.6875
Query ID 290/475
Query set metrics: Loss = 0.7557, accuracy = 0.6875, precision = 0.7619, recall = 0.8333, F1 score = 0.7381
Query ID 291/475
Query set metrics: Loss = 1.2893, accuracy = 0.6875, precision = 0.6389, recall = 0.6759, F1 score = 0.6204
Query ID 292/475


Query set metrics: Loss = 1.0423, accuracy = 0.6875, precision = 0.7875, recall = 0.7292, F1 score = 0.7444
Query ID 351/475
Query set metrics: Loss = 0.6802, accuracy = 0.7500, precision = 0.7037, recall = 0.7130, F1 score = 0.7026
Query ID 352/475
Query set metrics: Loss = 1.3263, accuracy = 0.5625, precision = 0.6146, recall = 0.6250, F1 score = 0.5667
Query ID 353/475
Query set metrics: Loss = 0.6857, accuracy = 0.8750, precision = 0.8021, recall = 0.8333, F1 score = 0.8071
Query ID 354/475
Query set metrics: Loss = 1.6093, accuracy = 0.5625, precision = 0.7500, recall = 0.6111, F1 score = 0.6222
Query ID 355/475
Query set metrics: Loss = 2.2492, accuracy = 0.5000, precision = 0.5417, recall = 0.4688, F1 score = 0.4940
Query ID 356/475
Query set metrics: Loss = 0.8853, accuracy = 0.7500, precision = 0.7100, recall = 0.7417, F1 score = 0.7133
Query ID 357/475
Query set metrics: Loss = 1.2974, accuracy = 0.7500, precision = 0.7593, recall = 0.7963, F1 score = 0.7370
Query ID 358/475


Query set metrics: Loss = 1.1311, accuracy = 0.7500, precision = 0.8571, recall = 0.8095, F1 score = 0.7762
Query ID 417/475
Query set metrics: Loss = 0.7582, accuracy = 0.8750, precision = 0.9167, recall = 0.9250, F1 score = 0.8990
Query ID 418/475
Query set metrics: Loss = 1.7080, accuracy = 0.6250, precision = 0.6458, recall = 0.6667, F1 score = 0.5917
Query ID 419/475
Query set metrics: Loss = 1.4632, accuracy = 0.5625, precision = 0.7083, recall = 0.5417, F1 score = 0.5429
Query ID 420/475
Query set metrics: Loss = 1.4179, accuracy = 0.6250, precision = 0.4630, recall = 0.5000, F1 score = 0.4593
Query ID 421/475
Query set metrics: Loss = 1.4599, accuracy = 0.6875, precision = 0.6208, recall = 0.6625, F1 score = 0.6375
Query ID 422/475
Query set metrics: Loss = 1.6669, accuracy = 0.6875, precision = 0.7708, recall = 0.6979, F1 score = 0.6863
Query ID 423/475
Query set metrics: Loss = 0.8097, accuracy = 0.8125, precision = 0.8333, recall = 0.8333, F1 score = 0.8042
Query ID 424/475


In [11]:
_model_path0 = os.path.splitext(model_path)[0]
csv_filename = _model_path0 + "_update"+ str(updates) +"_results.csv" # for selective replay
with open(csv_filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["data_idx", "label", "label_conf", "pred"])
    csv_writer.writerows(data_for_visual)
print(f"Done writing CSV File at {csv_filename}")

Done writing CSV File at /data/model_runs/original_oml/aMEL-allquery-order4/OML-order4-id4-2023-10-02_08-07-55.558316_update0_results.csv


In [12]:
# Log Time for Inference
_model_path0 = os.path.splitext(model_path)[0]
time_txt_filename = _model_path0 + "_update"+ str(updates) +"_time_inference.csv" 
with open(time_txt_filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["time_id", "time"])
    csv_writer.writerow(["Total Time", f"{toc//60} minutes"])
    csv_writer.writerow(["mean Adapt Time", f"{np.mean(all_adapt_time)} s"])
print(f"Done writing Time CSV File at {time_txt_filename}")

Done writing Time CSV File at /data/model_runs/original_oml/aMEL-allquery-order4/OML-order4-id4-2023-10-02_08-07-55.558316_update0_time_inference.csv
